In [ ]:
import requests
from bs4 import BeautifulSoup
import os

# 1.Scrapping the link of each page and connect them with the categories

In [ ]:

url = "https://time.com/vault/subject/influence/"

# Make a GET request to the page
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')

div1 = soup.find("div", class_="clearfix vault__header")
#print(div1)

category_divs = div1.find_all("div", class_="vault__collapsible__content")[1]
#print()

secondary_category_divs = category_divs.find("div", class_="vault__collapsible__content__secondary")
#print(secondary_category_divs)


# The topics as provided
topics = [
    "arts", "business", "government", "influence",
    "military", "politics", "science", "society",
    "sports", "world"
]

# Dictionary to hold categories and their links
categories = {topic: [] for topic in topics}

# Loop through each divtag and assign URLs to the corresponding topic in the dictionary
for topic, divtag in zip(topics, secondary_category_divs):
    atags = divtag.find_all("a", class_="vault__collapsible__secondary vault__collapsible__item")
   # Iterate over each 'a' tag to get its 'href'
    for atag in atags:
        url = atag['href']
        # Convert the link to a global link
        urlAGlobal = "https://time.com" + url
        categories[topic].append(urlAGlobal)
        #print(urlAGlobal)
#print(categories)


# 2.Function to download an image and return its local path

In [ ]:
import os
import requests
import pandas as pd


# Define the base directory to save the images
#BASE_DIR = "/Users/sutingting/Documents/DH project/time magazine"

BASE_DIR = "/Users/sutingting/Documents/DH project/gender"

# Initialize a global counter to ensure unique filenames
image_counter = 1

# Function to download an image and return its local path
def download_image(img_url):
    global image_counter  # Reference the global counter

    # Generate a new filename for each image as "img01.jpg", "img02.jpg", etc.
    local_filename = f"img{image_counter:02}.jpg"
    image_counter += 1  # Increment the counter for the next image

    # Construct the full local filepath
    local_filepath = os.path.join(BASE_DIR, local_filename)

    # Send a GET request to the image URL
    response = requests.get(img_url, stream=True)
    if response.status_code == 200:
        # Make sure the base directory exists, create it if not
        os.makedirs(os.path.dirname(local_filepath), exist_ok=True)
        # Open the file in binary write mode and write the contents of the response
        with open(local_filepath, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

    # Return the filepath where the image was saved
    return local_filepath

In [ ]:
import os
import requests
import pandas as pd

# Define the base directory to save the images
BASE_DIR = "/Users/sutingting/Documents/DH project/gender"

# Function to download an image and return its local path, named after image_index
def download_image(row):
    img_url = row['img_url']
    image_index = row['image_index']

    # Generate a new filename for each image as "image01.jpg", "image02.jpg", etc., based on image_index
    local_filename = f"image{image_index:02}.jpg"

    # Construct the full local filepath
    local_filepath = os.path.join(BASE_DIR, local_filename)

    # Send a GET request to the image URL
    response = requests.get(img_url, stream=True)
    if response.status_code == 200:
        # Make sure the base directory exists, create it if not
        os.makedirs(os.path.dirname(local_filepath), exist_ok=True)
        # Open the file in binary write mode and write the contents of the response
        with open(local_filepath, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
    else:
        print(f"Failed to download {img_url}")
        return None  # Return None if download fails

    # Return the filepath where the image was saved
    return local_filepath

# Load the CSV file
csv_file_path = '/Users/sutingting/Documents/DH project/Time_gender_modified.csv'
df = pd.read_csv(csv_file_path)

# Ensure 'image_index' is an integer for consistent filename formatting
df['image_index'] = df['image_index'].astype(int)

# Download images and store their local file paths in a new column
df['local_image_path'] = df.apply(download_image, axis=1)

# save the DataFrame with the new 'local_image_path' column to a new CSV
df.to_csv('/Users/sutingting/Documents/DH project/Time_gender_with_local_paths.csv', index=False)

print("Images have been downloaded and DataFrame updated.")


Images have been downloaded and DataFrame updated.


# 3. Crawling image and meta data

In [ ]:
pip install selenium

In [ ]:
pip install webdriver-manager

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import os
import time
from bs4 import BeautifulSoup

In [ ]:
# Setup Chrome with Selenium and the Chrome WebDriver
options = Options()
options.headless = True  # Run in headless mode
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")


service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

In [ ]:
# Dictionary to store the results
results = []

# Avoid overloading the server
request_delay = 1.0

# Iterate through each category and its links
for category, links in categories.items():

    for url in links:
        # Navigate to the page using Selenium
        driver.get(url)
        # Wait for the content to load (adjust time as necessary)
        time.sleep(request_delay)

        # Once the page is loaded, get the page source
        page_content = driver.page_source
        soup = BeautifulSoup(page_content, 'html.parser')

        # Find articles by class name "vault__item"
        articles = soup.find_all("article", class_="vault__item")
        #print(articles)


        for article in articles:
            # For each article, find the image and the date
            image_tag = article.find("img")
            date_tag = article.find("span", class_="vault__item__date")

            if image_tag and date_tag:
                # Get the image URL and the date
                img_url = image_tag['src'].split('?')[0] #without query parameters
                date_text = date_tag.text.strip()

                 # Download the image and get the local path
                ##local_image_path = download_image(img_url)

                # Add the information to the results dictionary
                results.append({
                    'category': category,
                    "date": date_text,
                    "img_url": img_url,
                    #'local_image_path': local_image_path
                })

# Quit the Selenium WebDriver
driver.quit()
# Print or process the scraped data
for result in results:
    print(result)

{'category': 'arts', 'date': 'April 7, 1923', 'img_url': 'https://api.time.com/wp-content/uploads/2014/11/15505.jpg'}
{'category': 'arts', 'date': 'July 30, 1923', 'img_url': 'https://api.time.com/wp-content/uploads/2014/11/155021.jpg'}
{'category': 'arts', 'date': 'Aug. 13, 1923', 'img_url': 'https://api.time.com/wp-content/uploads/2014/11/155023.jpg'}
{'category': 'arts', 'date': 'Sept. 17, 1923', 'img_url': 'https://api.time.com/wp-content/themes/time2014/img/vault/nocover--small.jpg'}
{'category': 'arts', 'date': 'Oct. 8, 1923', 'img_url': 'https://api.time.com/wp-content/themes/time2014/img/vault/nocover--small.jpg'}
{'category': 'arts', 'date': 'Nov. 5, 1923', 'img_url': 'https://api.time.com/wp-content/uploads/2014/11/155034-e1430862107116.jpg'}
{'category': 'arts', 'date': 'Dec. 17, 1923', 'img_url': 'https://api.time.com/wp-content/uploads/2014/11/155040-e1430862101319.jpg'}
{'category': 'arts', 'date': 'Dec. 24, 1923', 'img_url': 'https://api.time.com/wp-content/uploads/2014/

# 4. data munging

In [ ]:
import pandas as pd

# Convert the results list to a DataFrame
df = pd.DataFrame(results)

# Extract the year from the 'date' column
df['year'] = df['date'].str.split(', ').str[-1]
df.drop('date', axis=1, inplace=True)

# Create an 'image_index' column starting from 1
df['image_index'] = df.index + 1

# Move the 'image_index' column to the first position
column_order = ['image_index'] + [col for col in df.columns if col != 'image_index']
df = df[column_order]

# Create a decade variable

# Define a function to convert the year to the corresponding decade category
def year_to_decade(year_str):
    year = int(year_str)
    # Find the start of the corresponding decade
    decade_start = (year // 10) * 10
    # Set the start year to the 1920s if it's less than 1930
    if decade_start < 1930:
        return "1920s"
    # Set the start year to the 2010s if it's greater than 2010
    elif decade_start > 2010:
        return "2010s"
    # For other years, simply add 's' to denote the decade
    else:
        return f"{decade_start}s"

# Apply the function to the 'year' column to convert and create a new 'decade' column
df['decade'] = df['year'].apply(year_to_decade)

# Display the updated DataFrame to confirm the existence of 'year' and 'decade' columns
print(df)



      image_index category                                            img_url  \
0               1     arts  https://api.time.com/wp-content/uploads/2014/1...   
1               2     arts  https://api.time.com/wp-content/uploads/2014/1...   
2               3     arts  https://api.time.com/wp-content/uploads/2014/1...   
3               4     arts  https://api.time.com/wp-content/themes/time201...   
4               5     arts  https://api.time.com/wp-content/themes/time201...   
...           ...      ...                                                ...   
4806         4807    world  https://api.time.com/wp-content/uploads/2014/1...   
4807         4808    world  https://api.time.com/wp-content/uploads/2014/1...   
4808         4809    world  https://api.time.com/wp-content/uploads/2014/1...   
4809         4810    world  https://api.time.com/wp-content/uploads/2014/1...   
4810         4811    world  https://api.time.com/wp-content/uploads/2014/1...   

      year decade  
0     1

In [ ]:
# Use the DataFrame groupby method along with the apply method to remove duplicates within each group
df_deduped = df.groupby('category', as_index=False).apply(lambda x: x.drop_duplicates(subset='img_url', keep='first')).reset_index(drop=True)

# Now, df_deduped contains the deduplicated DataFrame
display(df_deduped)

csv_file_path = "/Users/sutingting/Documents/DH project/Time_Unique.csv"  # Specify the path to save the CSV file
df_deduped.to_csv(csv_file_path, index=False)  # Save the DataFrame to a CSV file, excluding the index

,image_index,category,img_url,year,decade
0,1,arts,https://api.time.com/wp-content/uploads/2014/1...,1923,1920s
1,2,arts,https://api.time.com/wp-content/uploads/2014/1...,1923,1920s
2,3,arts,https://api.time.com/wp-content/uploads/2014/1...,1923,1920s
3,4,arts,https://api.time.com/wp-content/themes/time201...,1923,1920s
4,6,arts,https://api.time.com/wp-content/uploads/2014/1...,1923,1920s
...,...,...,...,...,...
3253,4806,world,https://api.time.com/wp-content/uploads/2014/1...,1982,1980s
3254,4807,world,https://api.time.com/wp-content/uploads/2014/1...,1982,1980s
3255,4808,world,https://api.time.com/wp-content/uploads/2014/1...,1982,1980s
3256,4809,world,https://api.time.com/wp-content/uploads/2014/1...,1982,1980s


In [ ]:
#check and count duplicates  optional

# First, make a copy of the original DataFrame to preserve it
df_new = df_deduped.copy()

# Count the occurrences of each 'img_url' in the new DataFrame
img_url_counts = df_new['img_url'].value_counts()

# Map the counts to the 'img_url' column to create a new 'img_url_count' column in the new DataFrame
df_new['img_url_count'] = df_new['img_url'].map(img_url_counts)

# Now, df_new includes a 'img_url_count' column showing how many times each img_url appears
print(df_new.head())
#csv_file_path = "/Users/sutingting/Documents/DH project/countcheck_after.csv"  # Specify the path to save the CSV file
#df_new.to_csv(csv_file_path, index=False)  # Save the DataFrame to a CSV file, excluding the index

   image_index category                                            img_url  \
0            1     arts  https://api.time.com/wp-content/uploads/2014/1...   
1            2     arts  https://api.time.com/wp-content/uploads/2014/1...   
2            3     arts  https://api.time.com/wp-content/uploads/2014/1...   
3            4     arts  https://api.time.com/wp-content/themes/time201...   
4            6     arts  https://api.time.com/wp-content/uploads/2014/1...   

   year decade  img_url_count  
0  1923  1920s              1  
1  1923  1920s              1  
2  1923  1920s              1  
3  1923  1920s              9  
4  1923  1920s              1  
